In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Conv2D, Flatten, concatenate
from tensorflow.keras.models import Model

# 1. subjects ID Embedding 

In [ ]:
# 데이터 불러오기

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/sub_id3.csv')

In [ ]:
# 사람_ID를 범주형 변수로 변환

label_encoder = LabelEncoder()
df['ID_encoded'] = label_encoder.fit_transform(df['sub_id'])

In [ ]:
# embedding layer 생성

import tensorflow as tf
from tensorflow.keras.layers import Embedding
print(tf.__version__)

n_unique_ids = df['ID_encoded'].nunique()
embedding_dim = min(n_unique_ids // 2, 50)  # 임베딩 차원 설정. 보통은 고유 ID의 수의 절반 혹은 50을 선택

embedding_layer = Embedding(input_dim=n_unique_ids, 
                            output_dim=embedding_dim, 
                            input_length=1, 
                            name='ID_embedding')

In [ ]:
# 임베딩 레이어를 통과하여 벡터 생성 후, Flatten 적용

from tensorflow.keras.layers import Flatten

# 데이터를 모델에 넣을 수 있는 형태로 변환
input_data = df['ID_encoded'].values.reshape(-1, 1)

# 임베딩 레이어를 통과
embedded_data = embedding_layer(input_data)

# Flatten the output to enable concatenation
sub_id_input = Flatten()(embedded_data)

# 2. video_type_OneHotEncoding

In [ ]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv('/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/video_id.csv')

In [ ]:
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)

# Perform one-hot encoding
one_hot_encoded = one_hot_encoder.fit_transform(df['video_ID'].values.reshape(-1, 1))

# Convert the result back to a dataframe
df_encoded = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.categories_[0])

# Convert df_encoded to a suitable input for Keras
video_id_input = df_encoded.values

# 3. EEG Signal Denoising AutoEncoder

In [1]:
#Packages & libraries
from scipy.io import loadmat
from scipy.signal import cwt, ricker
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from concurrent.futures import ThreadPoolExecutor
from math import ceil
import gc

eeg_folder_path = '/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/EEG'
sub_id_path = '/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/sub_id2.csv'

In [2]:
from sklearn.preprocessing import MinMaxScaler

def load_and_scale_eeg_data(eeg_folder_path, sub_id_path, batch_size=6):
    # Load sub_id
    sub_id_df = pd.read_csv(sub_id_path)
    sub_id_list = sub_id_df['sub_id'].apply(lambda x: f"{x:03d}").tolist()

    max_length = 0
    eeg_data = []
    for sub_id in sub_id_list:
        mat_file_path = os.path.join(eeg_folder_path, f"{sub_id}.mat")
        if os.path.exists(mat_file_path):
            mat = loadmat(mat_file_path)
        else:
            print(f"Error: File not found for sub_id {sub_id}")
            continue

        for segment in ['Segment_3', 'Segment_6', 'Segment_9', 'Segment_12', 'Segment_15', 'Segment_18']:
            eeg_segment = mat[segment]
            max_length = max(max_length, eeg_segment.shape[1])  # update max_length
            eeg_data.append(eeg_segment)

        print(f"Processed data for sub_id {sub_id}")

    # Zero padding
    for i in range(len(eeg_data)):
        if eeg_data[i].shape[1] < max_length:
            zero_pad = np.zeros((eeg_data[i].shape[0], max_length - eeg_data[i].shape[1]))
            eeg_data[i] = np.hstack((eeg_data[i], zero_pad))

    # Convert list of arrays into 3D array
    eeg_data = np.stack(eeg_data)

    # Min-Max Scaling
    scaler = MinMaxScaler()
    eeg_data = eeg_data.reshape(-1, eeg_data.shape[-1])  # collapse the first two dimensions
    eeg_data = scaler.fit_transform(eeg_data)  # scale the data
    eeg_data = eeg_data.reshape(-1, max_length, 10)  # reshape back into original shape

    # Split the data into batches
    num_batches = eeg_data.shape[0] // batch_size
    eeg_data = eeg_data[:num_batches*batch_size]  # ignore last few samples if not enough for a batch
    eeg_data = eeg_data.reshape(num_batches, batch_size, max_length, 10)

    return eeg_data

eeg_data = load_and_scale_eeg_data(eeg_folder_path, sub_id_path, batch_size=6)
eeg_data = eeg_data.astype(np.float32)  # change the data type to float32

Processed data for sub_id 001
Processed data for sub_id 005
Processed data for sub_id 006
Processed data for sub_id 009
Processed data for sub_id 010
Processed data for sub_id 011
Processed data for sub_id 012
Processed data for sub_id 013
Processed data for sub_id 015
Processed data for sub_id 021
Processed data for sub_id 022
Processed data for sub_id 023
Processed data for sub_id 024
Processed data for sub_id 026
Processed data for sub_id 028
Processed data for sub_id 031
Processed data for sub_id 032
Processed data for sub_id 033
Processed data for sub_id 036
Processed data for sub_id 038
Processed data for sub_id 041
Processed data for sub_id 042
Processed data for sub_id 044
Processed data for sub_id 045
Processed data for sub_id 047
Processed data for sub_id 053
Processed data for sub_id 054
Processed data for sub_id 055
Processed data for sub_id 060
Processed data for sub_id 063
Processed data for sub_id 064
Processed data for sub_id 066
Processed data for sub_id 070
Processed 

In [3]:
print("Data type of eeg_data:", eeg_data.dtype)

Data type of eeg_data: float32


In [6]:
# Assume that 'eeg_data_scaled' is a list of EEG data arrays
X = np.array(eeg_data)  # convert list to numpy array
X = X.reshape(-1, *X.shape[2:])  # remove batch dimension and make it 3D

# Adding Gaussian Noise
noise_factor = 0.5
X_noisy = X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X.shape).astype(np.float32)

In [7]:
print(X.dtype)
print(X_noisy.dtype)
print(X.shape[1])
print(X.shape[2])

float32
float32
78008
10


In [ ]:
# 모델 아키텍처에 투입해서 특징 추출

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D

# Shape of single EEG data array
n_time_steps, n_channels = X.shape[1], X.shape[2]

# Input layer
eeg_input = Input(shape=(n_time_steps, n_channels), name='eeg_input', dtype=tf.float32)  # specify the data type of Input layer

# Encoding
x = Conv1D(16, 3, activation='relu', padding='same')(eeg_input)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
encoded = MaxPooling1D(2, padding='same')(x)

# Decoding
x = Conv1D(16, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
x = UpSampling1D(2)(x)
decoded = Conv1D(n_channels, 3, activation='sigmoid', padding='same')(x)

# Build the autoencoder model
autoencoder = Model(inputs=eeg_input, outputs=decoded)

# Build the feature extraction model (encoder part of the autoencoder)
feature_extractor = Model(inputs=eeg_input, outputs=encoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder with noisy data as input and clean data as output
autoencoder.fit(X_noisy, X, epochs=30, batch_size=16, shuffle=True, verbose=2)

# Extract features using the feature_extractor model
features = feature_extractor.predict(X_noisy)

# Flatten the features
features_flattened = features.reshape(features.shape[0], -1)

# Save the flattened features
np.save('features.npy', features_flattened)

# Check the shape and content of features
print(features_flattened.shape)
print(features_flattened)

Metal device set to: Apple M2 Pro
Epoch 1/30


2023-06-06 11:03:03.581209: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# Concatenate (#1 + #2 + #3)

In [ ]:
from keras.layers import Concatenate

# Concatenate the output from the different branches
concatenated = Concatenate()([sub_id_output, video_id_output, eeg_output])